# Curso de Word2Vec: treinamento de Word Embedding

In [34]:
import pandas as pd

# 1. Explorando o Spacy

In [35]:
df_treino = pd.read_csv('/home/franciscofoz/Documents/GitHub/machine-learning-training/Processamento de linguagem natural/Datasets/w2v_treino.csv')
df_teste = pd.read_csv('/home/franciscofoz/Documents/GitHub/machine-learning-training/Processamento de linguagem natural/Datasets/w2v_teste.csv')

In [36]:
df_treino

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
...,...,...,...,...,...,...
89995,"Mural: Há 30 anos, aeroporto não foi bem receb...",Década de 1970. Congonhas já estava superlotad...,2015-01-22,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/01...
89996,"As notícias sobre Schumacher não são boas, diz...",O ex-presidente da Ferrari Luca di Montezemolo...,2016-04-02,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/02/1...
89997,"De olho em R$ 50 bilhões, governo pode concede...","Para fazer caixa, o governo estuda conceder pa...",2017-08-29,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
89998,Moro deu a Lula o papel de coitadinho,Realizou-se parcialmente o primeiro objetivo d...,2016-06-03,colunas,eliogaspari,http://www1.folha.uol.com.br/colunas/eliogaspa...


In [37]:
import spacy

nlp = spacy.load('pt_core_news_sm')

In [38]:
texto = 'Rio de Janeiro é uma cidade maravilhosa'
doc = nlp(texto)

In [39]:
type(doc)

spacy.tokens.doc.Doc

In [40]:
doc[0]

Rio

In [41]:
type(doc[0])

spacy.tokens.token.Token

In [42]:
doc.ents

(Rio de Janeiro,)

In [43]:
type(doc.ents)

tuple

In [44]:
doc[1]

de

In [45]:
doc[1].is_stop

True

# 2. Pré processamento com Spacy

In [46]:
texto_para_tratamento = (titulos.lower() for titulos in df_treino.title)

In [47]:
def trata_textos(doc):
    
    tokens_validos = list()
    
    for token in doc:
        eh_valido = not token.is_stop and token.is_alpha
        if eh_valido:
            tokens_validos.append(token.text)
            
    if len(tokens_validos) > 2:
        return ' '.join(tokens_validos)
    

In [48]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [49]:
from time import time

t0 = time()

textos_tratados = [trata_textos(doc) for doc in nlp.pipe(texto_para_tratamento,batch_size=1000,n_process= -1)]

tf = time() - t0

print(tf/60)

1.8640616059303283


In [50]:
titulos_tratados = pd.DataFrame({'titulos':textos_tratados})

In [51]:
titulos_tratados

,titulos
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc
...,...
89995,mural há anos aeroporto recebido moradores gua...
89996,notícias schumacher boas ferrari
89997,olho bilhões governo conceder áreas petróleo
89998,moro deu lula papel coitadinho


# 3. Hiperparâmetros do Word2Vec

In [52]:
from gensim.models import Word2Vec


w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.007)

In [53]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [54]:
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados.titulos]

In [55]:
import logging

logging.basicConfig(format='%(asctime)s : - %(message)s',level = logging.INFO)

w2v_modelo.build_vocab(lista_lista_tokens,progress_per=5000)




2024-04-30 19:34:49,147 : - collecting all words and their counts
2024-04-30 19:34:49,149 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-04-30 19:34:49,160 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-04-30 19:34:49,176 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-04-30 19:34:49,189 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-04-30 19:34:49,200 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-04-30 19:34:49,213 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word types
2024-04-30 19:34:49,226 : - PROGRESS: at sentence #30000, processed 191554 words, keeping 25494 word types
2024-04-30 19:34:49,240 : - PROGRESS: at sentence #35000, processed 223412 words, keeping 27330 word types
2024-04-30 19:34:49,252 : - PROGRESS: at sentence #40000, processed 255282 words, keeping 29053

# 4. Treinamento do Word2Vec

In [56]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30)

2024-04-30 19:34:49,776 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2024-04-30T19:34:49.776170', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.5.0-28-generic-x86_64-with-glibc2.35', 'event': 'train'}
2024-04-30 19:34:50,530 : - EPOCH 0: training on 540242 raw words (486188 effective words) took 0.7s, 658429 effective words/s
2024-04-30 19:34:51,264 : - EPOCH 1: training on 540242 raw words (486151 effective words) took 0.7s, 668481 effective words/s
2024-04-30 19:34:52,235 : - EPOCH 2: training on 540242 raw words (486229 effective words) took 1.0s, 504016 effective words/s
2024-04-30 19:34:53,249 : - EPOCH 3 - PROGRESS: at 100.00% examples, 484598 words/s, in_qsize 0, out_qsize 1
2024-04-30 19:34:53,250 : - EPOCH 3: training on 540242 raw words (486034 effective words) took 1.0s, 4837

(14584461, 16207260)

In [57]:
w2v_modelo.wv.most_similar('revista')

[('playboy', 0.4001206159591675),
 ('processada', 0.3645883798599243),
 ('hugh', 0.3630234897136688),
 ('capa', 0.3609415888786316),
 ('aliança', 0.3601062297821045),
 ('abbey', 0.3542258143424988),
 ('anônimos', 0.3495810329914093),
 ('got', 0.3494630455970764),
 ('allen', 0.345334529876709),
 ('depardieu', 0.3414243459701538)]

In [58]:
w2v_modelo.wv.most_similar('livro')

[('biografia', 0.4514586627483368),
 ('romance', 0.4380078613758087),
 ('antologia', 0.43577247858047485),
 ('virginia', 0.43511706590652466),
 ('livros', 0.4023575782775879),
 ('houellebecq', 0.4016284942626953),
 ('memórias', 0.3980974853038788),
 ('autobiografia', 0.39702656865119934),
 ('coletânea', 0.3951440453529358),
 ('cd', 0.3836880028247833)]

In [59]:
w2v_modelo.wv.most_similar('coca')

[('cultivo', 0.5860814452171326),
 ('gastronomia', 0.45166119933128357),
 ('publicados', 0.4361380338668823),
 ('homicídios', 0.434289813041687),
 ('culinária', 0.4296986758708954),
 ('termômetro', 0.4270356595516205),
 ('computadores', 0.41144418716430664),
 ('assinantes', 0.40799835324287415),
 ('firmam', 0.4076526463031769),
 ('puxada', 0.4010506868362427)]

In [60]:
w2v_modelo.wv.most_similar('gato')

[('orson', 0.52555251121521),
 ('paixão', 0.4877203404903412),
 ('viúva', 0.4702746868133545),
 ('genial', 0.46040648221969604),
 ('brincalhão', 0.45456767082214355),
 ('achei', 0.44957220554351807),
 ('neta', 0.4491339325904846),
 ('banal', 0.44517019391059875),
 ('dela', 0.44451817870140076),
 ('cocaína', 0.44066253304481506)]

# 4. Treinamento do modelo

In [ ]:
import numpy as np
import spacy
from gensim.models import KeyedVectors



In [61]:
nlp = spacy.load('pt_core_news_sm', disable=['parser','ner','tagger','textcat'])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = list()
    for token in doc:
        eh_valido = not token.is_stop and token.is_alpha
        if eh_valido:
            tokens_validos.append(token.text.lower())
    
    return tokens_validos

In [62]:
texto = 'Rio de Janeiro 12213323 ***** @#$ é uma cidade maravilhosa'

In [63]:
tokenizador(texto)

['rio', 'janeiro', 'cidade', 'maravilhosa']

In [ ]:


def combinacao_de_vetores_por_soma(palavras,modelo):
    
    vetor_resultante = np.zeros(300)
    
    for pn in palavras:
        
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass
                
    return vetor_resultante
    

In [ ]:
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    
    matriz = np.zeros((x,y))
    
    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)
        
    return matriz